# Workshop No. 3

As you are becoming an expert on understand systems, it is the time to create an __expert system__.

Also, as you shoud know, you professor love movies a lot. So, you want to help him to choose next movies to watch. In this sense, you will take information for next movies from a popular website called ___IMDb___ where there is different information about movies and opinions.

However, you do not want to spend money to get movies information using a _webAPI_. But, as you are a good _computer engineer_, you know there is a technique called _web scrapping_, and you could get information using _HTML_ from an website, and put this information into a _DataFrame_ just to make easier data processing.

In this case, you are gonna use _request_ package to get HTML code from ___IMDb___ site, _beautiful soup_ package to make web scrapping, and _pandas_ to have a dataframe to handle data.

In [ ]:
# neccesary libraries
!pip install --upgrade pip
!pip install requests
!pip install beautifulsoup4
!pip install pandas

## 1. Get Movies Data

In the link [IMDb Upcoming Relases](https://www.imdb.com/calendar/?ref_=rlm&region=US&type=MOVIE) there are next movies with some related data as genre or star actors. So, using html tags, you could take information and save it into python variables. At the end, here you want to have a DataFrame with movies information.

In [9]:
# import libraries
import requests
from bs4 import BeautifulSoup

def get_web_html(url: str) -> BeautifulSoup:
    """
    This method gets the HTML from a website using scrapping.

    Args:
        url: URL to scrape.

    Returns:
        A BeautifulSoup object with the HTML.
    """
    # headers to avoid 403 error, cos' IMDb blocks requests from bots
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    # get HTML and save it in a variable
    response = requests.get(url, headers=headers)
    return BeautifulSoup(response.text, "html.parser")


def get_director_writer(url: str) -> tuple:
    """
    This method gets the director and writer from a movie page on IMDb.

    Args:
        url: URL of the movie page.

    Returns:
        A tuple with director and writer.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    print(soup)

    # Find director and writer
    director = soup.find_all('div', class_='credit_summary_item')[0].find('a').text
    writer = soup.find_all('div', class_='credit_summary_item')[1].find('a').text

    return director, writer

url = "https://www.imdb.com/title/tt21261712/?ref_=rlm"
director, writer = get_director_writer(url)
#print(f"Director: {director}")
#print(f"Writer: {writer}")

<!DOCTYPE html>
<html lang="en-US" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/"><head><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><script>if(typeof uet === 'function'){ uet('bb', 'LoadTitle', {wb: 1}); }</script><script>window.addEventListener('load', (event) => {
        if (typeof window.csa !== 'undefined' && typeof window.csa === 'function') {
            var csaLatencyPlugin = window.csa('Content', {
                element: {
                    slotId: 'LoadTitle',
                    type: 'service-call'
                }
            });
            csaLatencyPlugin('mark', 'clickToBodyBegin', 1715548014824);
        }
    })</script><title>Back to Black (2024) - IMDb</title><meta content="Back to Black: Directed by Sam Taylor-Johnson. With Marisa Abela, Jack O'Connell, Eddie Marsan, Lesley Manville. The life and music of Amy Winehouse, through the journey of adolescence to adulthood and the creation

IndexError: list index out of range

In [ ]:
# import libraries
from bs4 import BeautifulSoup


def get_title_year(movie_data: BeautifulSoup) -> tuple:
    """
    This methog gets the title, year, and link of a movie data using scrapping.

    Args:
        movie_data: BeautifulSoup object with movie data.

    Returns:
        A tuple with title, year, and link.
    """
    # process main title of subdivission
    base_url = "https://www.imdb.com"
    ref_data = movie_data.find("a")
    title = ref_data.text.split("(")[0].strip()
    year = ref_data.text.split("(")[1].replace(")", "")
    link = base_url + ref_data["href"]
    return title, year, link


def get_genre_actors(movie_data: BeautifulSoup) -> list:
    """
    This method gets genre and actors from a movie data using scrapping.

    Args:
        movie_data: BeautifulSoup object with movie data.

    Returns:
        A list with genre and actors.
    """
    # process ul tags to get genre and actors
    data = []
    ul_tags = movie_data.find_all("ul")
    for ul_tag in ul_tags:
        temp = []
        for li_tag in ul_tag.find_all("li"):
            temp.append(li_tag.find("span").get_text())
        data.append(", ".join(temp))
    return data

In [7]:
# import libraries
from bs4 import BeautifulSoup
import pandas as pd


def generate_dataframe(url: str) -> pd.DataFrame:
    """
    This method generates a DataFrame with movie data from IMDb.

    Args:
        url: URL to scrape.

    Returns:
        A DataFrame with movie data.
    """
    # movies datastructure definition
    imdb = get_web_html(url)
    movies = []
    movies_metadata = ["Title", "Year", "Genre", "Actors"]

    # process HTML using scrapping going to each division with the class ipc-metadata-list-summary-item__tc
    movies_html = imdb.find_all("div", class_="ipc-metadata-list-summary-item__tc")
    for movie in movies_html:
        # get each movie data into a clean html structure
        movie_data = BeautifulSoup(str(movie), "html.parser")

        # get movie data
        title, year, link = get_title_year(movie_data)
        data = get_genre_actors(movie_data)

        # create a dictionary to have a nice data movie structure
        movie_clean_data = {
            "Title": title,
            "Year": year,
            "Genre": data[0],
            "Actors": data[1] if len(data) > 1 else "",
        }
        # create a list of dictionaries to create a DataFrame
        movies.append(movie_clean_data)

    # create movies dataframe
    return pd.DataFrame(movies, columns=movies_metadata)
    
# ================================ MAIN =================================== #
# url to scrape
url = "https://www.imdb.com/calendar/?ref_=rlm&region=US&type=MOVIE"
movies_df = generate_dataframe(url)
print(movies_df.head(3))
    

                      Title  Year                     Genre  \
0             Back to Black  2024   Biography, Drama, Music   
1                        IF  2024  Animation, Comedy, Drama   
2  The Strangers: Chapter 1  2024                    Horror   

                                              Actors  
0  Marisa Abela, Eddie Marsan, Jack O'Connell, Le...  
1  Ryan Reynolds, John Krasinski, Cailey Fleming,...  
2  Madelaine Petsch, Gabriel Basso, Rachel Shento...  


## 2. Time to build a Decision Tree

In order to create an expert system, you must define a decision tree. It means, a sequence of conditionals in order to get a result, it is like to build a flowchart composed for a lot of conditionals.

In this part, you must to go to any site like _draw.io_ and create a __flowchart__ about how you think is the best decision process: _what questions do you want to make?_

Remember, make the right questions is a pretty important step at any taks you want to do.

As follows is the algorithm used in the ___expert system___ (replace image called _expert\_system.png_):

![Expert System](expert_system.png)


### 2.1 Your proposal

_In this section you must write your system analysis, and justify the questions selection and order. Also, define system components, system sensibility, anything you consider important to solve this problem._

## 3. Put your expert system at work

Now it is the time to write code in order to create your first and __classical expert system__. Here it is important to use conditionals, and based on conditionals apply filters in _pandas dataframe_.

![Classical AI](classical_ai.jpg)

In [46]:
# HERE write your code

***

# Some Useful Examples

I know you are learning about _python_, _coding_, and this technologies. As follows there are some simple examples about how you can use filtering in _pandas_.

In [5]:
# filter by an exact value
filtered_movies_df = movies_df[movies_df['Year'] == '2023']
print(filtered_movies_df.head(3))

          Title  Year                      Genre  \
5  The American  2023                      Drama   
8        Rapito  2023             Drama, History   
9         Sight  2023  Biography, Drama, History   

                                              Actors  
5  Talia Ryder, Diane Kruger, Oleg Ivenko, Natash...  
8  Paolo Pierobon, Fausto Russo Alesi, Barbara Ro...  
9  Greg Kinnear, Fionnula Flanagan, Terry Chen, B...  


In [6]:
# filter by a string option
filtered_movies_df = movies_df[movies_df['Genre'].str.contains('Thriller', na=False)]
print(filtered_movies_df.head(3))

                  Title  Year                      Genre  \
12        Terra Infirma  2024  Action, Thriller, Western   
13  In a Violent Nature  2024    Drama, Horror, Thriller   
18       April 29, 1992  2024    Action, Drama, Thriller   

                                               Actors  
12  Bruce Greenwood, Wes Studi, Adam Beach, Gísli ...  
13  Ry Barrett, Andrea Pavlovic, Cameron Love, Ree...  
18  Scott Eastwood, Ray Liotta, Tyrese Gibson, Dyl...  
